## Merge database tables into a flat dataframe

In [3]:
import sqlite3
import pandas as pd
# import spacy
# import sys

pd.set_option("display.max_colwidth", 60)
pd.set_option("display.max_rows", 100)

In [4]:
RELEASE_DB_NAME = "release_db.sqlite"
conn = sqlite3.connect(RELEASE_DB_NAME)

### Read in the tables

In [5]:

policy_texts_df = pd.read_sql_query("SELECT * FROM policy_texts", conn)


In [6]:
policy_texts_df.tail()

,id,policy_text,flesch_kincaid,smog,flesch_ease,length,sha1,simhash
333981,333982,Introduction\n\nCassava Enterprises (Gibraltar) Limited ...,15.972146,16.322122,difficult,11953,bebc0f148f8a8e62d63903bbf367b01e4ddfd6f8,5331428010802882003
333982,333983,Privacy policy This privacy policy applies only to this ...,14.503463,NaN,difficult,5582,33fc2ba320eac1bdfac5b0da0b095109c29c8b28,16403546848911186076
333983,333984,"Last updated: September 16, 2014\n\nOnestop Internet, In...",14.287103,15.774803,difficult,16906,fb8828ae29968ee6555c6238214aef2dd959fbad,13418782305040989778
333984,333985,84 Lumber Website Privacy Policy\n\nThe 84 Lumber Compan...,13.354206,NaN,difficult,4168,7de55ba3018ceee82880d4709c302fbd24fd5713,17414741223863505154
333985,333986,* About index\n * Contact us\n * Search tips\n * St...,16.554437,18.704411,very_confusing,18984,4b175a2504cbfc485c496f555687b868b7ff386d,15429291216000498142


In [7]:
simply_the_text = policy_texts_df[["id", "policy_text"]]
simply_the_text.head()

,id,policy_text
0,1,DocuSign Express Privacy Policy (11/03)\n\nThe privacy p...
1,2,DocuSign Express™ Privacy Policy (11/03)\n\nOverview\n\n...
2,3,The privacy practices of this statement apply to our ser...
3,4,Overview \n\nThe privacy practices of this statement app...
4,5,The privacy practices of this statement apply to our ser...


In [8]:
simply_the_text = simply_the_text[:500]

print(len(simply_the_text))

500


In [9]:
# use labels.json to label policies in simply_the_text dataframe
import json
with open('labels.json') as f:
    labels = json.load(f)
# create a new column in simply_the_text dataframe called "label"
simply_the_text["label"] = ""

simply_the_text.head()

# for each policy in simply_the_text dataframe, find the corresponding label in labels.json
for i in range(len(simply_the_text)):
    for title, labels_list in labels.items():
        # if any word in labels_list appears in policy_text, then add the title in the labels. there can be multiple labels for each policy
        if any(word in simply_the_text["policy_text"][i] for word in labels_list):
            simply_the_text["label"][i] +=  ", " + title if simply_the_text["label"][i] != "" else title

simply_the_text.head()

/var/folders/np/gp8d_33j4jxd2pd79hf5w03r0000gn/T/ipykernel_12146/2061048215.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simply_the_text["label"][i] +=  ", " + title if simply_the_text["label"][i] != "" else title
/var/folders/np/gp8d_33j4jxd2pd79hf5w03r0000gn/T/ipykernel_12146/2061048215.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simply_the_text["label"][i] +=  ", " + title if simply_the_text["label"][i] != "" else title
/var/folders/np/gp8d_33j4jxd2pd79hf5w03r0000gn/T/ipykernel_12146/2061048215.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

,id,policy_text,label
0,1,DocuSign Express Privacy Policy (11/03)\n\nThe privacy p...,"business, law, regulations, technology, multidisciplinary"
1,2,DocuSign Express™ Privacy Policy (11/03)\n\nOverview\n\n...,"business, law, regulations, technology, multidisciplinary"
2,3,The privacy practices of this statement apply to our ser...,"business, law, regulations, technology, multidisciplinary"
3,4,Overview \n\nThe privacy practices of this statement app...,"business, law, regulations, technology, multidisciplinary"
4,5,The privacy practices of this statement apply to our ser...,"business, law, regulations, technology, multidisciplinary"


In [10]:
import json
import pandas as pd

with open('labels.json') as f:
    labels = json.load(f)

# Create a new column in `simply_the_text` dataframe called "annotations"
simply_the_text["annotations"] = ""

# Iterate over each policy in `simply_the_text` dataframe
for i, row in simply_the_text.iterrows():
    policy_text = row["policy_text"]
    annotations = []
    seen_tokens = set()  # Track the tokens that have been seen
    
    # Iterate over each label in labels.json
    for label, label_keywords in labels.items():
        # Check if any word in `label_keywords` appears in `policy_text`
        for keyword in label_keywords:
            start = policy_text.find(keyword)
            while start != -1:
                end = start + len(keyword)
                if all(token not in seen_tokens for token in range(start, end)):
                    annotations.append((start, end, label))
                    seen_tokens.update(range(start, end))
                start = policy_text.find(keyword, end)
    
    # Add the annotations to the "annotations" column
    simply_the_text.at[i, "annotations"] = {"entities": annotations}

# Display the updated dataframe
print(simply_the_text)


      id                                                  policy_text   
0      1  DocuSign Express Privacy Policy (11/03)\n\nThe privacy p...  \
1      2  DocuSign Express™ Privacy Policy (11/03)\n\nOverview\n\n...   
2      3  The privacy practices of this statement apply to our ser...   
3      4  Overview \n\nThe privacy practices of this statement app...   
4      5  The privacy practices of this statement apply to our ser...   
..   ...                                                          ...   
495  496  Effective from 29th May 2019\n\nwww.retailchoice.com is ...   
496  497  RETURN PATH PRIVACY STATEMENT\n\n|   \n  \n---  \n  \nWe...   
497  498    \nReturn Path's Privacy Promise  \nWe take privacy ver...   
498  499  Effective Date: 08/07/2014  \nPosting Date: 08/07/2014\n...   
499  500  Revenue fully respects your right to privacy in relation...   

                                                         label   
0    business, law, regulations, technology, multidiscipl

In [11]:
simply_the_text.head()

,id,policy_text,label,annotations
0,1,DocuSign Express Privacy Policy (11/03)\n\nThe privacy p...,"business, law, regulations, technology, multidisciplinary","{'entities': [(6141, 6150, 'business'), (11910, 11915, '..."
1,2,DocuSign Express™ Privacy Policy (11/03)\n\nOverview\n\n...,"business, law, regulations, technology, multidisciplinary","{'entities': [(6228, 6237, 'business'), (12072, 12077, '..."
2,3,The privacy practices of this statement apply to our ser...,"business, law, regulations, technology, multidisciplinary","{'entities': [(6169, 6178, 'business'), (12074, 12079, '..."
3,4,Overview \n\nThe privacy practices of this statement app...,"business, law, regulations, technology, multidisciplinary","{'entities': [(6189, 6198, 'business'), (12100, 12105, '..."
4,5,The privacy practices of this statement apply to our ser...,"business, law, regulations, technology, multidisciplinary","{'entities': [(6202, 6211, 'business'), (12105, 12110, '..."


In [13]:
import pandas as pd
import spacy
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example

train_data = simply_the_text.sample(frac=0.8, random_state=42)
test_data = simply_the_text.drop(train_data.index)
nlp = spacy.blank("en")  # Use an appropriate spaCy model

# Prepare the training data in the required format for NER
train_data_ner = []
for _, row in train_data.iterrows():
    doc = nlp.make_doc(row["policy_text"])
    annotations = row["annotations"]
    
    example = Example.from_dict(doc, annotations)
    train_data_ner.append(example)

# Train the NER model
print("Training the NER model...")

# Add the NER component to the pipeline
if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Add custom entity labels to the NER pipeline
labels = ['business', 'law', 'regulations', 'usability', 'education', 'technology', 'multidisciplinary']
for label in labels:
    ner.add_label(label)

# Disable other pipeline components for training efficiency
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    batch_sizes = compounding(4.0, 32.0, 1.001)
    epochs = 10

    for epoch in range(epochs):
        random.shuffle(train_data_ner)
        losses = {}
        batches = minibatch(train_data_ner, size=batch_sizes)
        print(f"Epoch {epoch+1}/{epochs}")
        for batch in batches:
            nlp.update(batch, drop=0.3, losses=losses)
        print(f"Loss: {losses['ner']:.4f}")

# Evaluate the model on the testing set
print("Evaluating the model on the testing set...")
test_data_ner = []
for _, row in test_data.iterrows():
    doc = nlp.make_doc(row["policy_text"])
    annotations = row["annotations"]
    example = Example.from_dict(doc, annotations)
    test_data_ner.append(example)

# Make predictions using the trained model
print("Making predictions using the trained model...")
predictions = []
for example in test_data_ner:
    doc = nlp(example.text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    predictions.append(entities)

# Evaluate the model's performance using suitable metrics
print("Model evaluation completed.")

# Save the trained model for future use
print("Saving the trained model...")
nlp.to_disk("trained_ner_model")
print("Model saved successfully.")


In [36]:
# use the model
import spacy
nlp = spacy.load("trained_ner_model")

# Define a function to predict labels for a given text
def predict_labels(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Define a function to display the predicted labels
def display_predictions(text):
    entities = predict_labels(text)
    for entity in entities:
        # find the last fullstop before the start of the entity
        entity_start = text.find(entity[0])
        entity_end = entity_start + len(entity[0])
        start = text.rfind(".", 0, entity_start) + 1
        # find the next fullstop after the end of the entity
        end = text.find(".", entity_end) + 1
        # print the sentence containing the entity
        print(f"{text[start:end].strip()} : {entity[1]}")
        

# Predict labels for the given text
# get the 501th policy text
text = policy_texts_df[["id", "policy_text"]]
display_predictions(text.iloc[501, 1])

We take every precaution to protect the confidentiality of your personal information in accordance with Data Protection legislation. : law
Examples are when you apply for a tax clearance certificate, request verification of a tax clearance certificate or apply for mortgage interest relief. : technology
